In [ ]:
import uproot as upr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob

In [ ]:
from ROOT import TLorentzVector 

In [ ]:
from src.plotting_tools.cms_format import cms_style, cms_format_fig
from src.plotting_tools.Bins import Bins, bins
from src.plotting_tools.SysHist import SysHist

In [ ]:
cms_style()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [ ]:
outdir = 'assets_nov_2022'

In [ ]:
#file_2016 = '/eos/cms/store/group/phys_exotica/bffZprime/private_samples/signal/2016_NanoAODv6/BFFZprimeToMuMu_M125p0_dbs_0p04_2016/BFFZprimeToMuMu_M125p0_dbs_0p04_2016.root'
#file_2017 ='/eos/cms/store/group/phys_exotica/bffZprime/private_samples/signal/2017_NanoAODv6/BFFZprimeToMuMu_M175p0_dbs_0p04_2017/BFFZprimeToMuMu_M175p0_dbs_0p04_2017.root'
#file_2018 = '/eos/cms/store/group/phys_exotica/bffZprime/private_samples/signal/2018_NanoAODv6/ZprimeToMuMu175_2018/ZprimeToMuMu175_2018.root'

In [ ]:
#file_2016 = '/eos/cms/store/group/phys_exotica/bffZprime/sample_files/signal/BFF_200_0p04_2016.root'
#file_2016 = '/eos/cms/store/group/phys_exotica/bffZprime/sample_files/signal/BFF_350_0p04_2016.root'


In [ ]:
mass_ranges = {}
mass_ranges[125] = [110,150]
mass_ranges[200] = [150,250]
mass_ranges[350] = [300,400]
mass = 125
mass_range = mass_ranges[mass]

In [ ]:
def get_skimmed_file(era):
    path = '/eos/cms/store/group/phys_exotica/bffZprime/nanoAODskimmed/crab_bffv2/{}/BFF*{}*/**/*.root'.format(era, mass)
    return glob.glob(path, recursive=True)[0]

In [ ]:
skimmed_file = get_skimmed_file(2016)
skimmed_file_2017 = get_skimmed_file(2017)
skimmed_file_2018 = get_skimmed_file(2018)

In [ ]:
upf = upr.open(skimmed_file)['Events']
upf17 = upr.open(skimmed_file_2017)['Events']
upf18 = upr.open(skimmed_file_2018)['Events']

In [ ]:
#upf = (upr.open(skimmed_file)['Events'].keys())
#upf17 = (upr.open(skimmed_file_2017)['Events'].keys())
#upf18 = (upr.open(skimmed_file_2018)['Events'].keys())
#in_file = np.unique(np.array([*upf,*upf17, *upf18]).flatten())
#len(in_file)

In [ ]:
#from src.assets.bff_skimmer import columns, make_columns
#columns2 = [make_columns(era, columns) for era in ["2016", "2017", "2018"]]
#print([len(c) for c in columns2])
#columns2flat = np.unique(np.array(columns2).flatten())
#columns2flat.shape

In [ ]:
#import collections
#print([item for item, count in collections.Counter(columns2).items() if count > 1])

In [ ]:
#intersection = np.intersect1d(columns2flat, in_file)

In [ ]:
#intersection

In [ ]:
#set_diff = np.setdiff1d(columns2flat, intersection)

In [ ]:
#set_diff#

In [ ]:
#set_diff.shape[0] + intersection.shape[0]

In [ ]:
#set_diff

In [ ]:
mcTruth_df = upf.arrays(['GenPart_phi', 'GenPart_pt', 'GenPart_eta', 'GenPart_mass', 'GenPart_pdgId', 'GenPart_status'], library='pd')

In [ ]:
mc_muons = mcTruth_df[(mcTruth_df.GenPart_status==1) & (abs(mcTruth_df.GenPart_pdgId)==13)]

In [ ]:
mcreco_df = upf.arrays(['Muon_pt', 'Muon_phi', 'Muon_eta', 'Muon_mass'], library='pd')

In [ ]:
muon_list = []
for i, row in mc_muons.groupby(level=0):
    mctr_tl1 = TLorentzVector()
    mctr_tl2 = TLorentzVector()
    mcre_tl1 = TLorentzVector()
    mcre_tl2 = TLorentzVector()
    muonTr1 = row.iloc[0]
    muonTr2 = row.iloc[1]
    try:
        if len( mcreco_df.loc[i]) < 2: continue
    except:
        continue
    mctr_tl1.SetPtEtaPhiM(muonTr1.GenPart_pt, muonTr1.GenPart_eta, muonTr1.GenPart_phi, 
                         muonTr1.GenPart_mass)
    mctr_tl2.SetPtEtaPhiM(muonTr2.GenPart_pt, muonTr2.GenPart_eta, muonTr2.GenPart_phi, 
                         muonTr2.GenPart_mass)
    
    onematch = 0
    for j, row in mcreco_df.loc[i].iterrows():
        mcre_tl1.SetPtEtaPhiM(row.Muon_pt, row.Muon_eta, row.Muon_phi, 
                         row.Muon_mass)
        reco_kin1 = [row.Muon_pt, row.Muon_eta, row.Muon_phi, row.Muon_mass]
        if mcre_tl1.DeltaR(mctr_tl1) < .01: 
            onematch = 1
            break
    for k, row in mcreco_df.loc[i].iterrows():
        #no duplicate matches
        if j==k: continue
        mcre_tl2.SetPtEtaPhiM(row.Muon_pt, row.Muon_eta, row.Muon_phi, 
                         row.Muon_mass)
        reco_kin2 = [row.Muon_pt, row.Muon_eta, row.Muon_phi, row.Muon_mass]
        if mcre_tl2.DeltaR(mctr_tl2) < .01: 
            twomatch = 1
            break  
            
    if onematch*twomatch==0: continue
    #calculate pt_res
            
    pt_res1 = 1-mcre_tl1.Pt()/mctr_tl1.Pt()
    pt_res2 = 1-mcre_tl2.Pt()/mctr_tl2.Pt()
    if abs(pt_res1)> .4: continue
    if abs(pt_res2)> .4: continue
        
    muon_list.append({"pt_res1": pt_res1,
                    "pt_res2": pt_res2,
                    "gen_pt1": muonTr1.GenPart_pt,
                    "gen_pt2": muonTr2.GenPart_pt,
                    "reco_kin1":reco_kin1,
                    "reco_kin2":reco_kin2})
    if i%100==0: print(i)
    if i>1e4: break

In [ ]:
df = pd.DataFrame(muon_list)

In [ ]:
def smear_func(pt_res, kin, gen_pt, smear):
    pt, eta, phi, m = kin
    pt_smear = (1-(pt_res*(1+smear)))*gen_pt
    tl = TLorentzVector()
    tl.SetPtEtaPhiM(pt_smear, eta, phi, m)
    return tl


In [ ]:
row = df.loc[0]

In [ ]:
def make_dilep(smear):
    masses = []
    for i, row in df.iterrows():
        tl1 = smear_func(row.pt_res1, row.reco_kin1, row.gen_pt1, smear)
        tl2 = smear_func(row.pt_res2, row.reco_kin2, row.gen_pt2, smear)
        dilep = (tl1+tl2).M()
        masses.append(dilep)
    return masses

In [ ]:
def make_hist(smear):
    masses = make_dilep(smear)
    return np.histogram(masses, bins = bins.bin_edges)[0]

In [ ]:
def chi2_func(count1, count2):
    selection =  count2>0
    return np.average(((count1-count2)**2/count2)[selection])

In [ ]:
smear_dict = {}
smear0 =  make_hist(0)
chi2=0
smear = 0
while chi2 < 1:
    smear+=1e-2
    smearn =  make_hist(smear)
    chi2 = chi2_func(smear0,smearn)
    
print(smear, chi2)

In [ ]:
smearn =  make_hist(smear)

In [ ]:
mass_range[0]

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,12))

smear0_hist = SysHist(smear0, smear0*0, smear0*0, smear0**.5, bins.bin_edges )
smear0_hist.reduce_range(bottom=mass_range[0], top=mass_range[1]).make_density_hist().normalize().draw(ax, label='no additional smearing')

smearn_hist = SysHist(smearn, smearn*0, smearn*0, smearn**.5, bins.bin_edges )
smearn_hist.reduce_range(bottom=mass_range[0], top=mass_range[1]).make_density_hist().normalize().draw(ax, label='additional smearing: {:.0f} \%'.format(smear*100), color='red')


ax.set_yscale('log')
cms_format_fig('2016', ax)
ax.legend()
fig.savefig('{}/pt_smearing_width_study/{}_gev_2016.pdf'.format(outdir, mass))

In [ ]:
mass0 = make_dilep(0)
massn = make_dilep(smear)
sigma0 = np.std(mass0)
sigman = np.std(massn)

In [ ]:
sigma0, sigman

In [ ]:
(sigman**2-sigma0**2)**.5,  (sigman**2-sigma0**2)**.5/sigma0